# Neurocombat TOPMRI and EDIS harmonized datasets

Note this must be run in the `neuron` or `neuron_go` environment

## import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

from neuroCombat import neuroCombat

sys.path.insert(0, '../../') # path to functions

import cvasl.harmony as har

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
#pd.read_csv(filename_mri)# Datasets for this work
EDIS_path = '../our_datasets/EDIS/'
MRI_path = '../our_datasets/StrokeMRI/'
TOP_path = '../our_datasets/TOP/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)
EDIS_file = os.path.join(EDIS_path, file_name)


TOP = pd.read_csv(TOP_file)
MRI = pd.read_csv(MRI_file)
EDE = pd.read_csv(EDIS_file)

In [ ]:
TOP = TOP.drop(['Site', 'ID'], axis= 1)
TOP.head(3)

In [ ]:
MRI = MRI.drop(['Site', 'ID'],axis = 1)
MRI.tail(3)

In [ ]:
TOPMRI = pd.concat([MRI ,TOP])
TOPMRI.head(3)

In [ ]:
EDE = EDE.drop(['Site', 'ID'],axis = 1)
EDE.tail(3)

In [ ]:
TOPMRI.shape

In [ ]:
both_togetherF, ftF, btF, feature_dictF, len1, len2 = har.prep_for_neurocombat(TOPMRI, EDE)

In [ ]:
#both_togetherF.head(3)

In [ ]:
# our_mri_data = pd.read_csv(filename_mri)
# our_top_data = pd.read_csv(filename_top)

In [ ]:
# # save off csv
# both_together.to_csv('both_top_mri_together.csv')

In [ ]:
# # make and save of csv of features only
# features_only = both_together[2:]
# #features_only.to_csv('features_only_top_mri.csv')

In [ ]:
ftF.to_csv('ftF_top_mri_insight.csv')

In [ ]:
data = np.genfromtxt('ftF_top_mri_insight.csv', delimiter=",", skip_header=1)
data = data[:, 1:]
data

In [ ]:
TOPMRI.head(3)

In [ ]:
both_togetherF.head(3)

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * len1
last_columns_as_two = [2] * len2
covars = {'batch':first_columns_as_one + last_columns_as_two,
          #'Site': both_togetherF.loc['Site',:].values.tolist(), 
          'Sex':both_togetherF.loc['Sex',:].values.tolist(),
           'Age':both_togetherF.loc['Age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [ ]:
# specifify sex as categorical
categorical_cols = ['Sex']
# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'batch'
our_continuous_col=['Age']
#Harmonization step:
data_combat = neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col_mine,
    continuous_cols=our_continuous_col,
    categorical_cols=categorical_cols)["data"]

# (n_dat, covars, batch_col, continuous_cols=continuous_cols,
#                                     categorical_cols=categorical_cols)

In [ ]:
neurocombat = pd.DataFrame(data_combat)
neurocombat

In [ ]:
topperF = har.make_topper(btF,'Age', 'Sex')

In [ ]:
bottom = neurocombat.reset_index(drop=False)
bottom = bottom.rename(columns={"index": "char"})
bottom.columns = topperF.columns

In [ ]:
back_together = pd.concat([topperF, bottom])
back_together = back_together.T
#back_together

In [ ]:
new_header = back_together.iloc[0] #grab the first row for the header
back_together.columns = new_header #set the header row as the df header
back_together = back_together[1:]
#back_together

In [ ]:
back_together.head(3)

In [ ]:
neuro_harm_edis =back_together.tail(len2)
neuro_harm_topmri =back_together.head(len1)

In [ ]:
neuro_harm_topmri_v_e = neuro_harm_topmri.rename(feature_dictF, axis='columns')
neuro_harm_edis = neuro_harm_edis.rename(feature_dictF, axis='columns')

In [ ]:
neuro_harm_edis = neuro_harm_edis.reset_index()
neuro_harm_edis = neuro_harm_edis.rename(columns={"index": "participant_id"})
neuro_harm_edis

Save off to harmonized csv files

In [ ]:
neuro_harm_topmri_v_e = neuro_harm_topmri_v_e.reset_index()
neuro_harm_topmri_v_e = neuro_harm_topmri_v_e.rename(columns={"index": "participant_id"})
neuro_harm_topmri_v_e

In [ ]:
neuro_harm_edis.to_csv('harm_results/neuro_harm_edis.csv')
neuro_harm_topmri_v_e.to_csv('harm_results/neuro_harm_topmri_v_i.csv')

In [ ]:
#neuro_harm_sabre.columns

In [ ]:
number_columns = ['Age', 'Sex', 'GM_vol', 'WM_vol', 'CSF_vol',
       'GM_ICVRatio', 'GMWM_ICVRatio', 'WMHvol_WMvol', 'WMH_count',
       'DeepWM_B_CoV', 'ACA_B_CoV', 'MCA_B_CoV', 'PCA_B_CoV', 'TotalGM_B_CoV',
       'DeepWM_B_CBF', 'ACA_B_CBF', 'MCA_B_CBF', 'PCA_B_CBF', 'TotalGM_B_CBF',]

# Data quality check

In [ ]:
TOPMRI

In [ ]:
neuro_harm_topmri_v_e

In [ ]:
neuro_harm_edis.isna().sum().sum()

In [ ]:
neuro_harm_topmri.isna().sum().sum()

In [ ]:
#neuro_harm_top.head(3)
(neuro_harm_edis[number_columns] < 0).sum()

In [ ]:
(neuro_harm_topmri_v_e[number_columns] < 0).sum()

## So this new variable (WMHvol_WMvol ) is again problematic, BUT ALSO MORE 

specifically:

DeepWM_B_CoV      
ACA_B_CoV         
MCA_B_CoV         
PCA_B_CoV         
TotalGM_B_CoV     
DeepWM_B_CoV     
DeepWM_B_CBF     

In [ ]:
 We will wait about logging it until we see other columns we may want to log across all datasets

In [ ]:
# mri_selection = MRI
# rename_dict = {
#     'Age': 'age_unharm', # will be the key on merge
#     'Sex':'sex_unharm',
#     'GM_vol' :'gm_vol_unharm',
#     'WM_vol' :'wm_vol_unharm',
#     'CSF_vol':'csf_vol_unharm',
#     'gm_ivc_ratio': 'gm_ivc_ratio_unharm',
#     'gmwm_ivc_ratio': 'gmwm_ivc_ratio_unharm',
#     'wmh_vol' :'wmh_vol_unharm',
#     'wmh_count' : 'wmh_count_unharm',
#     'deepwm_b_cov':'deepwm_b_cov_unharm',
#     'aca_b_cov': 'aca_b_cov_unharm',
#     'mca_b_cov': 'mca_b_cov_unharm',
#     'pca_b_cov': 'pca_b_cov_unharm',
#     'totalgm_b_cov':'totalgm_b_cov_unharm',
#     'deepwm_b':'deepwm_b_unharm',
#     'aca_b':'aca_b_unharm',
#     'mca_b':'mca_b_unharm',
#     'pca_b':'pca_b_unharm',
#     'totalgm_b':'totalgm_b_unharm', 

# }
# mri_selection = mri_selection.rename(columns = rename_dict)
# mri_selection = mri_selection.merge(neuro_harm_mri, how= 'inner',)
# mri_selection

In [ ]:
# top_selection = TOP

# top_selection = top_selection.rename(columns = rename_dict)
# top_selection = top_selection.merge(neuro_harm_top, how= 'inner',)
# top_selection.columns

In [ ]:
# top_selection.to_csv('top_before_after_neurocombat2way.csv')
# mri_selection.to_csv('mri_before_after_neurocombat2way.csv')

Here we will also save off a version without white matter columns...optional

In [ ]:
# neuro_harm_mri_less = neuro_harm_mri.drop(columns=['wmh_vol', 'wmh_count'])
# neuro_harm_top_less = neuro_harm_top.drop(columns=['wmh_vol', 'wmh_count'])
# neuro_harm_mri_less.to_csv('less_neuro_harm_mri_mon.csv')
# neuro_harm_top_less.to_csv('less_neuro_harm_top_mon.csv')

# Investigate neurocombat results

In [ ]:
# neuro_harm_mri.head(3) 

## We see neuroharmoney added negative numbers 


## let's see how many negative numbers we have

In [ ]:
# number_columns = ['gm_vol',
#                   'wm_vol',
#                   'csf_vol',
#                   'gm_ivc_ratio', 
#                   'gmwm_ivc_ratio',
#                   'wmh_vol',
#                   'wmh_count',
#                   'deepwm_b_cov',
#                   'aca_b_cov',
#                   'mca_b_cov',
#                   'pca_b_cov',
#                   'totalgm_b_cov',
#                   'deepwm_b',
#                   'aca_b',
#                   'mca_b',
#                   'pca_b',
#                   'totalgm_b',]

In [ ]:
# (neuro_harm_mri[number_columns] < 0).sum()

In [ ]:
# #our_top_data.head(3)
# big_top = our_top_data.rename(columns={"Unnamed: 0": "level_0"})
# big_top.head(3)

In [ ]:
# new_harm_top = neuro_harm_top.reset_index()
# new_harm_top = new_harm_top.reset_index()
# new_harm_top = new_harm_top.rename(columns={"index": "participant_id"})
# new_harm_top.head(3) 

In [ ]:
# # difference in harmonized and original - raw
# raw_differences = big_top[number_columns] - new_harm_top[number_columns]

In [ ]:
# # difference in harmonized and original - raw
# differences = (big_top[number_columns] - new_harm_top[number_columns])/big_top[number_columns]
# differences

In [ ]:
# differences_n = differences.apply(pd.to_numeric) #

In [ ]:
# differences_n.describe()

## Now we can use the versions of TOP and StrokeMRI with log base 10 revision of white matter hyperintensity count, as well as white matter hyperintensity  volume... must be discussed with scientsis

In [ ]:
# log_filepath = '../open_work/internal_results/loggy/' 
# # this needs to be redone, due to bad column
# log_filename_mri = os.path.join(log_filepath,'stroke_loged_mon.csv') 
# log_filename_top = os.path.join(log_filepath,'top_loged_mon.csv') 

In [ ]:
#log_mri_data = pd.read_csv(log_filename_mri)
#log_top_data = pd.read_csv(log_filename_top)

In [ ]:
# log_both_togetherF, log_ftF, log_btF, log_feature_dictF, log_len1, log_len2 = prep_for_neurocombat(
#     log_filename_mri,
#     log_filename_top)

In [ ]:
# # make and save of csv of features only
# log_features_only = log_both_together[2:]
# #log_features_only.to_csv('log_features_only_top_mri.csv')

In [ ]:
# log_ftF.to_csv('log_ftF_top_mri.csv')

In [ ]:
# log_data = np.genfromtxt('log_ftF_top_mri.csv', delimiter=",", skip_header=1)
# log_data = log_data[:, 1:]

In [ ]:
# # Specifying the batch (scanner variable) as well as a sex covariate to preserve:
# first_columns_as_one = [1] * log_len1
# last_columns_as_two = [2] * log_len2
# covars = {'batch':first_columns_as_one + last_columns_as_two,
#           'sex':log_both_togetherF.loc['sex',:].values.tolist(),
#            'age':log_both_togetherF.loc['age',:].values.tolist(),} 
# covars = pd.DataFrame(covars) 

In [ ]:
# # specifify sex as categorical
# categorical_cols = ['sex']
# # To specify the name of the variable that encodes for the scanner/batch covariate:
# batch_col_mine = 'batch'

# #Harmonization step:
# log_data_combat = neuroCombat(dat=log_data,
#     covars=covars,
#     batch_col=batch_col_mine,
#     continuous_cols=our_continuous_col,
#     categorical_cols=categorical_cols)["data"]

In [ ]:
log_data_combat

In [ ]:
log_neurocombat = pd.DataFrame(log_data_combat)
#log_neurocombat

In [ ]:
log_topperF = make_topper(log_btF,'age', 'sex')

In [ ]:
log_bottom = log_neurocombat.reset_index(drop=False)
log_bottom = log_bottom.rename(columns={"index": "char"})
log_bottom.columns = log_topperF.columns
#log_bottom

In [ ]:
log_back_together = pd.concat([log_topperF, log_bottom])
log_back_together = log_back_together.T
#log_back_together

In [ ]:
log_new_header = log_back_together.iloc[0] #grab the first row for the header
log_back_together.columns = new_header #set the header row as the df header
log_back_together = log_back_together[1:]
#log_back_together

In [ ]:
log_back_together.head(514).tail(5)

In [ ]:
log_back_together.tail(527).head(5)

In [ ]:
log_neuro_harm_top =log_back_together.tail(log_len2)
log_neuro_harm_mri =log_back_together.head(log_len1)

In [ ]:
log_neuro_harm_top = log_neuro_harm_top.rename(log_feature_dictF, axis='columns')
log_neuro_harm_mri = log_neuro_harm_mri.rename(log_feature_dictF, axis='columns')

In [ ]:
(log_neuro_harm_mri[number_columns] < 0).sum()

In [ ]:
## There are still negative numbers in the outcome...but there is a difference

In [ ]:
# log_neuro_harm_mri.to_csv('log_neuro_harm_mri_mon.csv')
# log_neuro_harm_top.to_csv('log_neuro_harm_top_mon.csv')

In [ ]:
log_neuro_harm_mri.head(3)

In [ ]:
neuro_harm_mri.head(3)

In [ ]:
dif_log_to_reg = log_neuro_harm_mri[number_columns] - neuro_harm_mri[number_columns]
dif_log_to_reg 

In [ ]:
dif_log_to_reg.sum().sum()

# So changing the two columns to theirlog made la difference in the harmarmonized outcomes...now let's look at how this plays out when we make models...